In [1]:
import tensorflow as tf
import numpy as np
from alexnet_model import AlexNet
import matplotlib.pyplot as plt

# Function to prepare data for training, validation, and testing
def train_alexnet():
    # Create data generators
    train_data = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,       # Normalize pixel values to the range [0,1]
        validation_split=0.2  # Reserve 20% of training data for validation
    )
    
    test_data = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    
    # Loading Training and Validation Data
    train_generator = train_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Train',
        batch_size=128,          # Number of images per batch
        class_mode='categorical',  # One-hot encoding of labels
        subset='training'         # Use the 80% training split
    )
    
    validation_generator = train_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Train',
        batch_size=128,
        class_mode='categorical',
        subset='validation'      # Use the 20% validation split
    )
    
    # Loading Test Data
    test_generator = test_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Test',
        batch_size=128,
        class_mode='categorical'
    )
    
    # Create AlexNet Model
    model = AlexNet(num_classes=50)
    
    # Optimizer (SGD)
    optimizer = tf.keras.optimizers.SGD( 
        learning_rate=0.1,  # Initial learning rate
        momentum=0.9,        # Accelerates convergence
        nesterov=True
    )
    
    # Compiling the model
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',  # Loss function
        metrics=['accuracy']             # Track accuracy
    )
    
    # Training the model
    history = model.fit(
        train_generator,
        epochs=1,
        validation_data=validation_generator,
        callbacks=[
            tf.keras.callbacks.LearningRateScheduler(
                lambda epoch: 0.01 if epoch < 10 else (0.001 if epoch < 15 else 0.0001)  # Dynamic LR adjustment
            ),
            tf.keras.callbacks.ModelCheckpoint(
                'alexnet_checkpoint.keras',  # Save checkpoints
                save_best_only=True,
                monitor='val_accuracy'
            )
        ]
    )
    
    # Save final weights
    model.save_weights('alexnet_final.weights.h5')
    
    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    return model, history

# Run the training
if __name__ == "__main__":
    model, history = train_alexnet()


Found 30381 images belonging to 50 classes.
Found 7558 images belonging to 50 classes.
Found 9485 images belonging to 50 classes.


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


238/238 ━━━━━━━━━━━━━━━━━━━━ 345s 1s/step - accuracy: 0.0223 - loss: 3.9048 - val_accuracy: 0.0397 - val_loss: 3.8347 - learning_rate: 0.0100
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 449ms/step - accuracy: 0.0433 - loss: 3.8267
Test Loss: 3.8219
Test Accuracy: 0.0433
